In [1]:
import env, acquire, wrangle
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy.stats as stats

# Exercises II 

In [5]:
telco_df = acquire.get_telco_db('telco_churn') # import telco df 

In [3]:
telco_df # look at it to get a picture of the data

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [4]:
telco_df = telco_df[telco_df['contract_type_id']== 3] # select out 2-year contracts

In [5]:
telco_df = telco_df[['customer_id','monthly_charges','tenure','total_charges']] # only use the columns we wanta

In [6]:
telco_df 

,customer_id,monthly_charges,tenure,total_charges
5348,0040-HALCW,20.40,54,1090.6
5349,0042-RLHYP,19.70,69,1396.9
5350,0057-QBUQH,25.10,43,1070.15
5351,0064-SUDOG,20.30,12,224.5
5352,0071-NDAFP,25.50,25,630.6
...,...,...,...,...
7038,9919-FZDED,84.10,72,5981.65
7039,9926-PJHDQ,76.80,72,5468.45
7040,9928-BZVLZ,49.85,12,552.1
7041,9993-LHIEB,67.85,67,4627.65


In [7]:
telco_df.info()# check info for dtypes and empty fields. Lack of a null means that the field is a space

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 5348 to 7042
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


In [8]:
telco_df.describe().T # right now, it will only show monthly_charges and tenure because theyre they only num types

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [9]:
telco_df['total_charges'] = telco_df.total_charges.str.replace(' ','0.00').astype('float64') # replace empty fields in total charges with 0, and convert to a float

In [10]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 5348 to 7042
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 66.2+ KB


In [11]:
telco_df.tenure.astype('float64') # change tenure to a float so it matches the other nums

5348    54.0
5349    69.0
5350    43.0
5351    12.0
5352    25.0
        ... 
7038    72.0
7039    72.0
7040    12.0
7041    67.0
7042    63.0
Name: tenure, Length: 1695, dtype: float64

In [6]:
# create a function to just clean the data
def telco_clean(telco_df):
    telco_df = telco_df[telco_df['contract_type_id'] == 3]
    telco_df = telco_df[['customer_id','monthly_charges','tenure','total_charges']]
    telco_df['total_charges'] = telco_df.total_charges.str.replace(' ','0.00').astype('float64')
    telco_df.tenure = telco_df.tenure.astype('float64')
    return telco_df

In [7]:
telco_clean(telco_df)# test it

,customer_id,monthly_charges,tenure,total_charges
5348,0040-HALCW,20.40,54.0,1090.60
5349,0042-RLHYP,19.70,69.0,1396.90
5350,0057-QBUQH,25.10,43.0,1070.15
5351,0064-SUDOG,20.30,12.0,224.50
5352,0071-NDAFP,25.50,25.0,630.60
...,...,...,...,...
7038,9919-FZDED,84.10,72.0,5981.65
7039,9926-PJHDQ,76.80,72.0,5468.45
7040,9928-BZVLZ,49.85,12.0,552.10
7041,9993-LHIEB,67.85,67.0,4627.65


In [3]:
wrangle.wrangle_telco() # combine it with the get_telco_db function to put it all in one step

,customer_id,monthly_charges,tenure,total_charges
5348,0040-HALCW,20.40,54.0,1090.60
5349,0042-RLHYP,19.70,69.0,1396.90
5350,0057-QBUQH,25.10,43.0,1070.15
5351,0064-SUDOG,20.30,12.0,224.50
5352,0071-NDAFP,25.50,25.0,630.60
...,...,...,...,...
7038,9919-FZDED,84.10,72.0,5981.65
7039,9926-PJHDQ,76.80,72.0,5468.45
7040,9928-BZVLZ,49.85,12.0,552.10
7041,9993-LHIEB,67.85,67.0,4627.65


# Exercises III

In [20]:
zillow_df = acquire.get_zillow_db()

In [21]:
zillow_df.head(3)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0


In [22]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [23]:
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


### Initial thoughts
- plenty of null values
- min values for bedroom and bathroom counts are 0, either they are nulls or there is some wierd entries
- plenty of rows are not complete, the counts for each value vary wildly 
- dtypes are all floats, so we wont have to convert anything. 

### Next steps?
- drop nulls
- drop dupes

In [24]:
def missing_values_table(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values, and the percent of that column that has missing values
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           "columns that have missing values.")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [25]:
missing_values_table(zillow_df)

Your selected dataframe has 7 columns.
There are 6columns that have missing values.


,Missing Values,% of Total Values
yearbuilt,9337,0.4
calculatedfinishedsquarefeet,8484,0.4
taxamount,4442,0.2
taxvaluedollarcnt,493,0.0
bedroomcnt,11,0.0
bathroomcnt,11,0.0


In [12]:
zillow_df.isnull().sum() # count out the nulls. If we drop all the nulls, we'd still have over 2 million entries

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [26]:
zillow_df = zillow_df.dropna()

In [27]:
zillow_df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [28]:
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2140235 non-null  float64
 1   bathroomcnt                   2140235 non-null  float64
 2   calculatedfinishedsquarefeet  2140235 non-null  float64
 3   taxvaluedollarcnt             2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


> Even after dropping, we still have 2.1 million rows to work with

In [29]:
zillow_df.to_csv('zillow.csv') # cache it to speed up import

In [30]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2140235.0,3.301419,0.932600,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2140235.0,2.240352,0.990549,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2140235.0,1863.193970,1221.754161,1.00,1258.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2140235.0,460641.625164,677157.635675,22.00,189166.00,328296.00,534606.00,90188462.00
yearbuilt,2140235.0,1960.967545,22.150563,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2140235.0,5616.711322,7814.562798,6.34,2540.85,4111.47,6411.93,1078101.87
fips,2140235.0,6048.309556,20.344910,6037.00,6037.00,6037.00,6059.00,6111.00


In [31]:
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [32]:
zillow_df.groupby(zillow_df.columns.tolist(),as_index=False).size() # check for duplicates

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,size
0,0.0,0.0,1.0,28091.0,1963.0,439.55,6037.0,1
1,0.0,0.0,10.0,8033.0,1984.0,221.25,6037.0,1
2,0.0,0.0,20.0,22.0,2005.0,24.92,6037.0,2
3,0.0,0.0,20.0,1105.0,2005.0,61.82,6037.0,1
4,0.0,0.0,20.0,1124.0,2005.0,62.05,6037.0,2
...,...,...,...,...,...,...,...,...
2130209,16.0,16.0,7121.0,975000.0,1987.0,11218.15,6037.0,1
2130210,18.0,0.0,26116.0,20260690.0,2002.0,243003.64,6037.0,1
2130211,18.0,18.0,9309.0,2325000.0,1986.0,29283.87,6037.0,1
2130212,18.0,18.0,11134.0,2353308.0,1986.0,29165.39,6037.0,1


In [34]:
zillow_df = zillow_df.drop_duplicates() # drop the duplicates

In [35]:
zillow_df.to_csv('zillow.csv')

In [2]:
wrangle.wrangle_zillow()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152855,3.0,2.5,1809.0,405547.0,2012.0,4181.10,6059.0
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
